In [1]:
from models import LlamaForCausalLMNew
from transformers import AutoTokenizer, LlamaConfig
import torch
from datasets import load_dataset

/usr/scratch/haokang/anaconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_id = "meta-llama/Llama-2-7b-hf"
# config = LlamaConfig.from_pretrained(
#     model_id,
#     use_auth_token=True,
#     token=None,
#     use_flash_attn=False,
# )
model_kwargs = {}
model_kwargs["torch_dtype"] = torch.float16
model_kwargs["device_map"] = "auto"
model_kwargs["token"] = None
model_kwargs["cache_dir"] = "../cache"

In [ ]:
model = LlamaForCausalLMNew.from_pretrained(
    model_id,
    # config=config,
    **model_kwargs,
    # cache_dir="../cache",
    # compress_config=None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=None,
    padding_side="left",
    model_max_length=256,
    use_fast=False,
    cache_dir="../cache",
)
tokenizer.pad_token = tokenizer.eos_token

In [2]:
eval_dataset = load_dataset("gsm8k", "main", split="test")
# print(eval_dataset[0])
with open(f"./prompt_original.txt", "r") as handle:
    prompt_cot = handle.read()
    
questions = eval_dataset[0]["question"]
answers = eval_dataset[0]["answer"]
prompts = [
            prompt_cot + "\nQuestion: " + question + "\n" for question in questions
        ]
inputs = tokenizer(
    prompts,
    return_tensors="pt",
    padding="longest",
    truncation=True,
)
print(len(prompts))

280


In [ ]:

inputs = inputs.to("cuda")
print(inputs["input_ids"].shape)

In [ ]:
generated = model(**inputs,use_cache=True)

In [ ]:
import torch
from compress_function import fake_dense_sparse_uniformquantization
tensor = torch.load("key_states.pt")
print(tensor.shape)
test_tensor = tensor.reshape(74,-1).float()
U, S, Vh = torch.linalg.svd(test_tensor)

results = fake_dense_sparse_uniformquantization(test_tensor, 4, 0.01)
error = test_tensor - results

U, S1, Vh = torch.linalg.svd(error)


max = S.max()
S = S / max
max1 = S1.max()
S1 = S1 / max1
print(S)
print(S1)
import torch
import matplotlib.pyplot as plt

# 创建一个1D张量
tensor_1d = torch.tensor([1, 3, 5, 2, 4])

# 获取张量中的数据
data = S.numpy()
data1 = S1.numpy()
# 创建X轴的数据
x = range(1, len(data)+1)

# 绘制折线图
plt.plot(x, data)
plt.plot(x, data1)
# 添加轴标签
plt.xlabel('Index')
plt.ylabel('Value')

# 显示图像
plt.show()

In [ ]:
import torch
import matplotlib.pyplot as plt

# 创建一个1D张量
tensor_1d = torch.tensor([1, 3, 5, 2, 4])

# 获取张量中的数据
data = S.numpy()

# 创建X轴的数据
x = range(1, len(data)+1)

# 绘制折线图
plt.plot(x, data)

# 添加轴标签
plt.xlabel('Index')
plt.ylabel('Value')

# 显示图像
plt.show()

In [ ]:
import torch
tensor = torch.load("key_states.pt")
print(tensor.shape)
test_tensor = tensor.reshape(74,-1).float()

results = fake_dense_sparse_uniformquantization(test_tensor, 4, 0.01)
error = test_tensor - results

In [ ]:
U, S, Vh = torch.linalg.svd(error)
print(S)

In [ ]:
import torch
import matplotlib.pyplot as plt

# 创建一个1D张量
tensor_1d = torch.tensor([1, 3, 5, 2, 4])

# 获取张量中的数据
data = S.numpy()

# 创建X轴的数据
x = range(1, len(data)+1)

# 绘制折线图
plt.plot(x, data)

# 添加轴标签
plt.xlabel('Index')
plt.ylabel('Value')

# 显示图像
plt.show()

In [ ]:
import torch
import matplotlib.pyplot as plt

# 创建一个2D张量
tensor_2d = torch.rand((3, 4))

# 将张量转换为NumPy数组
array_2d = tensor[0,0,:,:].numpy()

# 使用matplotlib绘制图像
plt.imshow(array_2d, cmap='Greens')
plt.colorbar()

# 添加轴标签
plt.xlabel('Columns')
plt.ylabel('Rows')

# 显示图像
plt.show()